Автокликер записывает щелчки мышью и действия клавиатуры.
Для записи используется блок ниже

In [1]:
#запись логов
def logi(fale_name):
    from pynput.mouse import Listener as Listener1
    from pynput.keyboard import Key
    from pynput.keyboard import Listener as Listener2

    import logging
    fale_name = fale_name+".txt"

    logging.basicConfig(filename=fale_name, level=logging.DEBUG, format='%(asctime)s; %(message)s')

    def on_click(x, y, button, pressed):
        if pressed:
            logging.info('{2};{0};{1};;'.format(x, y, button))

    def on_scroll(x, y, dx, dy):
        logging.info('scroll;{0};{1};{2};{3}'.format(x, y, dx, dy))

    def on_press(key):
        logging.info("press.{0};;;;".format(key))
        if key == Key.esc:
            return False

    def on_release(key):

        logging.info("releas.{0};;;;".format(key))
        if key == Key.esc:
            return False


    with Listener1(on_click=on_click, on_scroll=on_scroll) as listener1, Listener2(on_press=on_press, on_release=on_release) as listener2:
        listener2.join()
        listener1.join()
       
    
logi('record') # record имя файла для записи логов в ТХТ

После записи логов в TXT файл можно удалить лишнее если было случайно записаны ненужные действия, удаляя строки целиком.

Ниже фунцкии обрабатывают файл логов и воспроизводят их.

In [ ]:
def obr_log(fale_name):

    fale_name = fale_name+".txt"
    df = pd.read_csv(fale_name, sep=';', names=[
                     'date', 'mouve', 'x', 'y', 'x_vector', 'y_vector'], encoding='ansi') 
    df = df.fillna(0)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S,%f')
    df['x'] = df['x'].astype(int)
    df['y'] = df['y'].astype(int)
    df['x_vector'] = df['x_vector'].astype(int)
    df['y_vector'] = df['y_vector'].astype(int)
    df['mouve'] = df['mouve'].str.replace('Button.', '')
    import datetime
    df['interval'] = 0
    for i in range(1, len(df)):
        df['interval'][i] = ((df['date'][i] - df['date'][i-1]).seconds)
    return df

def vosproizvesti_logi(df):
    import pyautogui as pag
    from PIL import Image
    import time
    import keyboard

    for i in range(len(df)):
        start = df['mouve'][i].find('.')
        do = df['mouve'][i]
        interval = df['interval'][i]
        if interval == 0:
            interval = 0.25
        elif interval > 10000:
            interval = 1
        elif interval > 10 and interval <= 10000:
            interval += 5
            # поднятие клавиши
        if do[1:start] == 'releas':
            # спецкнопка
            if do[start+1:start+4] == 'Key':
                time.sleep(interval)
                bitt = do[start+5:]
                pag.keyUp(bitt)
                # обычная кнопка
            else:
                time.sleep(interval)
                bitt = do[start+2:-1]
                pag.keyUp(bitt)
              # нажатие на клавишу
        elif do[1:start] == 'press':
            # спецкнопка
            if do[start+1:start+4] == 'Key':
                time.sleep(interval)
                bitt = do[start+5:]
                pag.keyDown(bitt)
                # обычная кнопка
            else:
                time.sleep(interval)
                bitt = do[start+2:-1]
                pag.keyDown(bitt)
                keyboard.write(bitt)
        elif do == ' left':
            time.sleep(interval)
            pag.click(x=df['x'][i], y=df['y'][i])

In [ ]:
vosproizvesti_logi(obr_log('record'))